# 실시간 Lip Landmark 추출

### 이전 실험과 바뀐 점
---

```실험```
- **Korea Dataset** 수집
- 모음별 Classification 및 Decoding

---

```데이터셋 수집시```
- **```첫번째 방법```**
    - **Video**로 수집하여, Frame 단위로 끊기
        - 기존: Frame 단위로 우선적 수집
    - Frame 단위로 끊을 시, **Window**를 정하여 overlapping을 통한 input 값 저장
        - 사람마다 다른 말의 속도 차이 무시 가능
        - 랜덤으로 augmentation도 가능

- **```두번째 방법```**
    - **LSTM** 관련 사용하여 입술의 벡터화 사용


---

```코드 흐름```
- 비디오로 저장하기
    - 비디오로 저장시, mouth가 추출되는지 안되는지만 확인
    - 없다면 Alert
- 비디오를 저장후, 랜덤으로 window 자르는 함수 구현
    - windowing(overlapping 추가) : augmentation
- 각 비디오 단위로 labeling 진행하기 [수작업]  

- 비디오가 잘못 잘린게 있다면 다시 자르기 [수작업]

---

```꼭 확인하고 실험하기```
- 5fps의 3초 길이 : 단어 하나 당


---


```실험 전```
- 사람&문장&반복횟수 **초기화**


----

```실험 동안```
- 코드 실행
- 코드에서 lip localization 한 값들이 뜨기 시작하면 시작
- 입을 벌리면 Frame 별 저장 시작
- 최대한 크게 최대한 가까이 최대한 밝은 공간에서 할 것!
- 얼굴은 정면으로
- 말은 조금은 느리게 (3초에 한단어씩이란 생각으로)
- 웹캠 끌때는 실행되고 있는 웹캠에서 'q' 눌러서 끄기
- 웹캠 반응이 없는데 계속 돌아가고 있다면, 위에 stop 눌러서 꺼주기
- 가끔 웹캠 반응 없으면 웹캠 종료 버튼 

---

```실험후```
- 사진들 저장되어있는거 확인
    - ***********No Detected******** 조심
- 앞이랑 뒤에 상관없는 사진들도 저장이 되었다면 삭제(입을 벌리기 직전 다물기 직전까지만 잘라서 저장: 수작업)
- crop사진들 확인 -> 얼굴 사진 저장되어있는 거랑 매치 시켜서 같이 저장되어있다면 **성공**
- 그렇지 않다면 **다시 찍기** -> 사진들 전부 폐기
- 결과 값 csv 파일로 저장

-  **커널 다시 돌리기**


---

```실험```
- 68개의 landmarks들을 사용해서 affine transformation 적용
    - to extract mouth-centered crop of size 100x50 pixel performance
- 전체 훈련 set에 걸쳐 RGB 채널을 평균 및 단위 분산이 0으로 표준화 시킴 (zero mean and unit variance)

---


## 입모양 확인
### 한글 기본 모음의 입모양은 '입술 모양, 입벌림, 발음의 전후차이'의 차이로 식별

- ```ㅏ```: 긴입술, 큰 입벌림, 입안 소리
- ```ㅓ```: 길다란 입술, 중간 입벌림, 입안소리
- ```ㅗ```; 둥근입술, 중간 입벌림, 입안 소리
- ```ㅜ```: 둥근입술, 작은 입벌림, 입안 소리
- ```ㅡ```: 길다란 입술, 작은 입벌림, 입안의 어금니 소리
-```ㅣ```: 길다란 입술, 작은 입벌림, 혀끝나온 앞니 소리

In [5]:
import pandas as pd
import dlib
import cv2
import numpy as np
import os

In [6]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [7]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x) # w
        rh = max(rh,y) # h
        rx = min(rx,x) # x
        ry = min(ry,y) # y
    return (rx,ry,rw-rx,rh-ry)

### person
|person|code|
|:--:|:--:|
|우정|00|
|희지|01|
|희수|02|
|영윤|03|
|은화|04|

### 문장
- 도와주세요
- 힘내세요
- 누구세요
- 안녕하세요
- 조심히가세요
- 죄송합니다
- 감사합니다
- 좋아요
- 싫어요

|앞말|끝말|
|:---|:---|
|도와주|세요|
|힘내|
|누구|
|안녕하|
|안녕히계|
|조심히가|
|죄송|합니다|
|감사|
|좋아|요|
|싫어|


|모음|단어|
|:---|:---|
|ㅏ|하 가 사 아 다|
|ㅏ2|안 감 합|
|ㅓㅕ|녕 어|
|ㅗㅛ|도 조 송 좋 요| 
|ㅜㅠ|주 누 구|
|ㅡㅣ|히 싫 니|
|ㅡㅣ2|힘 심|
|ㅐㅔㅖ|내 계 세|
|ㅘ|와|
|ㅚ|죄|

### 코드

|음절|코드|
|---|---|
|가|00|
|다|01|
|사|02|
|아|03|
|하|04|
|감|05|
|안|06|
|합|07|
|어|08|
|녕|09|
|도|10|
|조|11|
|송|12|
|좋|13|
|요|14|
|구|15|
|누|16|
|주|17|
|니|18|
|히|19|
|싫|20|
|심|21|
|힘|22|
|계|23|
|내|24|
|세|25|
|와|26|
|죄|27|

총 ```28단어```

In [14]:
######################################
# person & sentences
person = '00' 
sentence= '00' 
num = '00' # 반복 횟수
#######################################
count = 0
dataset = {}
dataset[person] = {}
dataset[person][sentence] ={}
dataset[person][sentence][num] ={}

# create face detector, predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") # predict 해주는애            

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_path = 'korea_data/%s/video' %(person)
try:
    os.makedirs(out_path)
except:
    pass
out_name = f'video_{sentence}_{num}.avi'
out_path = out_path + '/' + out_name
out = cv2.VideoWriter(out_path, fourcc, 5.0, (640, 480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        image = cv2.resize(frame, dsize = (640, 480), interpolation = cv2.INTER_AREA)
        
        img = image.copy()
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        out.write(image)
        
        face_detector = detector(img_gray, 1)
        if len(face_detector) == 0:
            print("***************No face Detected*************,%d", int(cap.get(1)))
        for face in face_detector:
            # cropping face
            crop_face = image.copy()
            crop_face = crop_face[face.top():face.bottom(), face.left():face.right()]
            # make prediction and transform to numpy array
            landmarks = predictor(image, face)
            landmarks = shape_to_np(landmarks)
            lip_landmarks=[landmarks[x] for x in range(48, 68)]
            image_crop = image.copy()
            (x, y, w, h) = get_rect(lip_landmarks)
            crop_image = image_crop[y:y+h, x:x+w]
            
            for(i,(x_i, y_i)) in enumerate(lip_landmarks):
                cv2.circle(image, (x_i, y_i), 1, (0,0,255), -1)
            
            (x, y, w, h) = get_rect(lip_landmarks)
            for s in lip_landmarks:
                s[0] -= x
                s[1] -= y
                
            
        cv2.imshow('frame',image)

        dataset[person][sentence][num][count] = {}
        if(int(cap.get(1))%3  == 0): 
            path = 'korea_data/%s/%s/%s/image' %(person, sentence, num)
            crop_path = 'korea_data/%s/%s/%s/crop_lip' %(person, sentence, num)
            crop_path2 = 'korea_data/%s/%s/%s/crop_face' %(person, sentence, num)
            
            # path 없다면 만들고 있다면 pass
            try:
                os.makedirs(path)
                os.makedirs(crop_path)
                os.makedirs(crop_path2)
            except FileExistsError as e:
                pass
            img_name = f'img_{count}.jpg'
            mouth_name = f'mouth_{count}.jpg'
            face_name = f'face_{count}.jpg'
            img_path = path + '/' + img_name
            mouth_path = crop_path + '/' + mouth_name
            face_path = crop_path2 + '/' + face_name
            
            # save images as frame
            print(img_path)
            print(img)
            
            cv2.imwrite(img_path, img)
            cv2.imwrite(mouth_path, crop_image)
            cv2.imwrite(face_path, crop_face)
            print(lip_landmarks)
            print('Saved Frame number: ' + str(int(cap.get(1)))+ '\n')
            dataset[person][sentence][num][count] = lip_landmarks
            count +=  1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

korea_data/00/00/00/image/img_0.jpg
[[[242 255 255]
  [242 255 255]
  [242 255 255]
  ...
  [186 185 189]
  [186 185 189]
  [187 186 190]]

 [[242 255 255]
  [242 255 255]
  [242 255 255]
  ...
  [186 185 189]
  [185 184 188]
  [186 185 189]]

 [[244 255 255]
  [244 255 255]
  [244 255 255]
  ...
  [187 187 189]
  [186 186 188]
  [186 186 188]]

 ...

 [[ 32  27  30]
  [ 24  19  22]
  [ 23  20  23]
  ...
  [140 128 121]
  [140 128 121]
  [140 128 121]]

 [[ 37  29  32]
  [ 27  20  23]
  [ 23  18  21]
  ...
  [143 133 126]
  [142 132 125]
  [141 131 124]]

 [[ 39  31  34]
  [ 28  21  24]
  [ 23  18  21]
  ...
  [143 133 126]
  [141 131 124]
  [139 129 122]]]
[array([ 0, 20]), array([14,  7]), array([28,  0]), array([38,  4]), array([48,  1]), array([63,  9]), array([77, 21]), array([63, 30]), array([48, 32]), array([38, 33]), array([27, 31]), array([14, 28]), array([ 7, 19]), array([27, 12]), array([37, 14]), array([47, 13]), array([71, 20]), array([48, 19]), array([38, 19]), array([28,

In [38]:
#dataset

In [39]:
# csv 파일은 문장 폴더 안에 입모양 반복 횟수대로 이름지어서 저장됨
fileCount = 0
shapes = []
for count in dataset[person][sentence][num].keys():
    shape = dataset[person][sentence][num][count]
    for (x,y) in shape:
        shapes.append((num,count,x,y))

df = pd.DataFrame(data = shapes,columns = ['num','frame','x','y'])
path = "csv"
try:
    os.makedirs(path)
except FileExistsError as e:
    pass
filename = "%s_%s_%s.csv" %(person, sentence, num)
csvPath = "%s/%s" % (path,filename)
print("saving %s" % csvPath)
df.to_csv(csvPath)

saving csv/M_10_02.csv


In [11]:
######################################
# person & sentences
person = 'TEST' 
sentence= '01' 
num = '01'
#######################################
count = 0
dataset = {}
dataset[person] = {}
dataset[person][sentence] ={}
dataset[person][sentence][num] ={}

import cv2

# create face detector, predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat") # predict 해주는애            

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_path = 'korea_data/%s/video' %(person)
try:
    os.makedirs(out_path)
except:
    pass
out_name = f'video_{sentence}_{num}.avi'
out_path = out_path + '/' + out_name
out = cv2.VideoWriter(out_path, fourcc, 5.0, (640, 480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        image = cv2.resize(frame, dsize = (640, 480), interpolation = cv2.INTER_AREA)
        
        img = image.copy()
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        out.write(image)
        
        face_detector = detector(img_gray, 1)
        if len(face_detector) == 0:
            print("***************No face Detected*************,%d", int(cap.get(1)))
        for face in face_detector:
            # cropping face
            crop_face = image.copy()
            crop_face = crop_face[face.top():face.bottom(), face.left():face.right()]
            # make prediction and transform to numpy array
            landmarks = predictor(image, face)
            landmarks = shape_to_np(landmarks)
            lip_landmarks=[landmarks[x] for x in range(48, 68)]
            image_crop = image.copy()
            (x, y, w, h) = get_rect(lip_landmarks)
            crop_image = image_crop[y:y+h, x:x+w]
            
            for(i,(x_i, y_i)) in enumerate(lip_landmarks):
                cv2.circle(image, (x_i, y_i), 1, (0,0,255), -1)
            
            (x, y, w, h) = get_rect(lip_landmarks)
            for s in lip_landmarks:
                s[0] -= x
                s[1] -= y
                
            
        cv2.imshow('frame',image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        dataset[person][sentence][num][count] = {}
        if(int(cap.get(1))%3  == 0): 
            path = 'korea_data/%s/%s/%s/image' %(person, sentence, num)
            crop_path = 'korea_data/%s/%s/%s/crop_lip' %(person, sentence, num)
            crop_path2 = 'korea_data/%s/%s/%s/crop_face' %(person, sentence, num)
            
            # path 없다면 만들고 있다면 pass
            try:
                os.makedirs(path)
                os.makedirs(crop_path)
                os.makedirs(crop_path2)
            except FileExistsError as e:
                pass
            img_name = f'img_{count}.jpg'
            mouth_name = f'mouth_{count}.jpg'
            face_name = f'face_{count}.jpg'
            img_path = path + '/' + img_name
            mouth_path = crop_path + '/' + mouth_name
            face_path = crop_path2 + '/' + face_name
            
            # save images as frame
            cv2.imwrite(img_path, img)
            cv2.imwrite(mouth_path, crop_image)
            cv2.imwrite(face_path, crop_face)
            print(lip_landmarks)
            print('Saved Frame number: ' + str(int(cap.get(1)))+ '\n')
            dataset[person][sentence][num][count] = lip_landmarks
            count +=  1
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

[array([ 0, 14]), array([9, 6]), array([19,  1]), array([27,  3]), array([35,  0]), array([46,  5]), array([56, 11]), array([47, 22]), array([37, 28]), array([28, 30]), array([20, 29]), array([10, 23]), array([ 5, 13]), array([19,  9]), array([27, 10]), array([35,  8]), array([51, 11]), array([36, 15]), array([28, 17]), array([20, 16])]
Saved Frame number: 24650175

[array([ 0, 18]), array([7, 7]), array([18,  1]), array([26,  3]), array([33,  0]), array([45,  6]), array([54, 15]), array([46, 32]), array([36, 40]), array([27, 42]), array([18, 41]), array([ 7, 34]), array([ 5, 18]), array([18, 10]), array([26, 10]), array([34,  9]), array([49, 16]), array([34, 24]), array([26, 26]), array([18, 25])]
Saved Frame number: 24650196

[array([ 0, 19]), array([7, 7]), array([18,  1]), array([25,  3]), array([33,  0]), array([45,  5]), array([53, 17]), array([45, 33]), array([35, 41]), array([26, 42]), array([18, 41]), array([ 7, 34]), array([ 5, 19]), array([18, 10]), array([26, 10]), array([3